# Publish your Collection using the EDC / BC Geoservice

This notebook demonstrates how to publish your collections as WMS and WFS service. The BC geoservice provides access to publishing your collection as a WMS service using the geoDB Python client. Please refer to the [geoserver](https://docs.geoserver.org/latest/en/user/services/wms/reference.html) documentation for configuring the rest API call to your WMS/geojson service.

You will have access to this service if you have purchased a large management account. If your account does not permit you to publish your dataset using the GeoServer (error message "Provided token doesn't have the required scope"), please request an upgraded account.
Also please note that this service is in beta mode. If you find any issues, please report them on our [GitHub Issues page](https://github.com/bcdev/xcube-hub/issues).

## Init the GeoDB client

In [ ]:
from xcube_geodb.core.geodb import GeoDBClient

geodb = GeoDBClient(gs_server_url='https://xcube-gen.brockmann-consult.de')
geodb._gs_server_url

## Create a Database and Copy some Public Data Across

In [ ]:
my_database_name = 'my_eurocrops'
if not geodb.database_exists(my_database_name):
    geodb.create_database(my_database_name)

Here we copy a collection from a public database, so we can publish as a geoservice. 

In [ ]:
my_collection = 'AT_2021_EC21'
geodb.copy_collection(collection=my_collection, new_collection=my_collection, database='geodb_b34bfae7-9265-4a3e-b921-06549d3c6035', new_database=my_database_name)

You can check if the colleciton was successfully added to your database:

In [ ]:
geodb.get_my_collections(database=my_database_name)

In [ ]:
## List Collections Published on the BC WMS Service

geodb.get_published_gs(database=my_database_name)

## Publish your Collection to the BC WMS Service

In [ ]:
geodb.publish_gs(collection=my_collection, database=my_database_name)

You can now click on the preview link which will give you an unstyled view of your WMS output.

In [ ]:
## List Collections Published on the BC WMS Service

geodb.get_published_gs(database=my_database_name)

## View the Data

Once you have published you collection, you can use the WMS service to visualise your collection. For this using ipyleaflet. In this example we have used a pre-defined style. You can always provide a custom style using the parameter `sld`. We have provided a class that will pass that parameter to the WMS service. Just replace WMSLayer with SldWMSLayer. The `sld`parameter is given as a URL to an SLD file.

In [ ]:
url = f"http://xcube-geodb.brockmann-consult.de/geoserver/{my_database_name}/wms?"


In [ ]:
from ipyleaflet import Map, WMSLayer, basemaps

In [ ]:
from traitlets import Unicode

class SldWMSLayer(WMSLayer):

    sld = Unicode('').tag(sync=True, o=True)


In [ ]:
wmsn = WMSLayer(
    url=url,
    layers=f'{my_database_name}:{my_database_name}_AT_2021_EC21',
    format='image/png',
    opacity=0.5,
    attribution='Copernicus',
    styles='ljubilana'
#     sld='https://raw.githubusercontent.com/dzelge/test-gha/main/ljubilana.sld'
)

In [ ]:
m = Map(basemap=basemaps.CartoDB.Positron, center=(46.0, 14.5), zoom=13)
m.add_layer(wmsn)

m

## Revert the changes made to the database

The following lines demonstrate how to remove the collection from the GeoServer again, and how to drop the collection from your database.

In [ ]:
geodb.unpublish_gs(collection=my_collection, database=my_database_name)

In [ ]:
geodb.drop_collection(my_collection, database=my_database_name)